# Concepts for Diagonalization of strain

This jupyter notebook is a worksheet that demostrates how coordinate transformation of strain works in the shear elastic modulus part works this program. You could clone this repo and change the `key` below and rerun it to see how the shear modulus solving mechanism is working for each of the elastic modulus

## Finding the transformation matrix

Define a shear elastic modulus $c_{ij}$ below as `key`

In [1]:
import numpy
from cij.util import c_

key = c_(15)

if not key.is_shear: raise RuntimeError(f"The strain c_{key} is not a shear modulus.")

Creating the corresponding fictitous strain $e$ for $c_{ij}$

In [2]:
e = numpy.zeros((3, 3))

e[key.i[0] - 1, key.i[1] - 1] = 1
e[key.i[1] - 1, key.i[0] - 1] = 1

e[key.j[0] - 1, key.j[1] - 1] = 1
e[key.j[1] - 1, key.j[0] - 1] = 1

print(e)

[[1. 0. 1.]
 [0. 0. 0.]
 [1. 0. 0.]]


The transformation matrix $T$ is the column-wise eigenvectors of the strain matrix $e$

In [3]:
evals, T = numpy.linalg.eig(e)
print(T)

[[ 0.85065081 -0.52573111  0.        ]
 [ 0.          0.          1.        ]
 [ 0.52573111  0.85065081  0.        ]]


The $e'$ in the rotated coordinate system is the matrix with eigenvalues of $e$ in its diagonals

In [4]:
print(numpy.diag(evals))

[[ 1.61803399  0.          0.        ]
 [ 0.         -0.61803399  0.        ]
 [ 0.          0.          0.        ]]


We can retrive the original strain with

$$
e' = T^{-1} e T
$$

In [5]:
print("e = ")
print(T @ numpy.diag(evals) @ T.T)

print("e = T e' T^{-1}:", numpy.allclose(e, T @ numpy.diag(evals) @ T.T))

e = 
[[ 1.00000000e+00  0.00000000e+00  1.00000000e+00]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00]
 [ 1.00000000e+00  0.00000000e+00 -5.55111512e-17]]
e = T e' T^{-1}: True


or backwards

$$
e = T e' T^{-1}
$$

In [6]:
print("e' = ")
print(T.T @ e @ T)

print("e' = T^{-1} e T:", numpy.allclose(numpy.diag(evals), T.T @ e @ T))

e' = 
[[ 1.61803399e+00  1.11022302e-16  0.00000000e+00]
 [ 1.66533454e-16 -6.18033989e-01  0.00000000e+00]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00]]
e' = T^{-1} e T: True


## Testing with symbols

In [7]:
import sympy

def round_expr(expr, num_digits=4):
    return expr.xreplace({n : round(n, num_digits) for n in expr.atoms(sympy.Number)})

### The strain used for calculating strain-Grüneisen parameter from volume-Grüneisen parameter

In [8]:
a = sympy.symbols("e_11")
b = sympy.symbols("e_22")
c = sympy.symbols("e_33")

m = sympy.Matrix([
    [a, 0, 0],
    [0, b, 0],
    [0, 0, c]
])

round_expr(T.T @ m @ T)

Matrix([
[ 0.7236*e_11 + 0.2764*e_33, -0.4472*e_11 + 0.4472*e_33,        0],
[-0.4472*e_11 + 0.4472*e_33,  0.2764*e_11 + 0.7236*e_33,        0],
[                         0,                          0, 1.0*e_22]])

In [9]:
a = sympy.symbols("α")
b = sympy.symbols("β")
c = sympy.symbols("γ")

m = sympy.Matrix([
    [1 + a, 0.   , 0.   ],
    [0.   , 1 + b, 0.   ],
    [0.   , 0.   , 1 + c]
])

round_expr(T.T @ m @ T)

Matrix([
[0.7236*α + 0.2764*γ + 1.0,      -0.4472*α + 0.4472*γ,           0],
[     -0.4472*α + 0.4472*γ, 0.2764*α + 0.7236*γ + 1.0,           0],
[                        0,                         0, 1.0*β + 1.0]])

### The strain energy invariance equation

The strain energy $E = \sum \frac{1}{2} c_{ijkl}e_{ij}e_{kl}$ should be same in the original and rotated coordinate system. Here we show the energy in both coordinate systems with the help of `sympy`.

In [10]:
import itertools
from cij.util import C_


def _meke_symbol(*argv):
    key = c_("".join(str(i+1) for i in argv))
    return sympy.symbols("c_%d%d" % key.v)

def get_strain_energy(fictitious_strain: numpy.ndarray):
    
    s = 0
      
    nz = numpy.nonzero(fictitious_strain)

    for (i, j), (k, l) in itertools.product(zip(*nz), zip(*nz)):
        e1 = fictitious_strain[i,j]
        e2 = fictitious_strain[k,l]
        c = _meke_symbol(i,j,k,l)
        s += c * e1 * e2
    
    return s

The **left hand side** of the strain energy invariance equation, which is under the **original coordinate system**, the expression for the strain $e$ defined above is

In [11]:
round_expr(get_strain_energy(e))

1.0*c_11 + 4.0*c_15 + 4.0*c_55

and on the **right hand side**, which is under the **rotated coordinate system**, the expression for the strain $e'$ defined above is

In [12]:
e_prime = numpy.diag(evals)
round_expr(get_strain_energy(e_prime))

2.618*c_11 - 2.0*c_12 + 0.382*c_22